<a href="https://colab.research.google.com/github/jdmartinev/ArtificialIntelligenceIM/blob/main/Lecture12/notebooks/L12_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
!pip install -q langchain
!pip install -q faiss-cpu
!pip install -q pypdf
!pip install openai
!pip install python-dotenv
!pip install tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 24.9 MB/s eta 0:00:00


In [62]:
!pip install -qU langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 30.5 MB/s eta 0:00:00


In [5]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.3 MB/s eta 0:00:00


In [13]:
cd '/content/drive/MyDrive/Courses/AI/Lecture12'

/content/drive/MyDrive/Courses/AI/Lecture12


In [89]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI

from pprint import pprint
import numpy as np
from openai import OpenAI
import os
from dotenv import load_dotenv
from tqdm.notebook import tqdm

In [65]:
load_dotenv('api_keys.env')
api_key = os.getenv('OPENAI_API_KEY')
# Create and retrieve a Cohere API key from os.cohere.ai
#co = cohere.Client(api_key)

In [41]:
def print_lines(text, max_ch = 50):
  current_line = ""
  words = text.split()
  i = 0
  while i < len(words):
    if len(current_line) > 50:
      print(current_line)
      current_line = ""
    else:
      current_line += f"{words[i]} "
      i+=1
  if current_line:
    print(current_line)

## 1. Datasets

In [16]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1YcNlffQl6E09Erst__EZvlC0iJRgy6LF' -O libros.zip
!unzip libros.zip -d ./libros

--2024-10-21 15:48:46--  https://docs.google.com/uc?export=download&id=1YcNlffQl6E09Erst__EZvlC0iJRgy6LF
Resolving docs.google.com (docs.google.com)... 142.251.10.138, 142.251.10.101, 142.251.10.100, ...
Connecting to docs.google.com (docs.google.com)|142.251.10.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1YcNlffQl6E09Erst__EZvlC0iJRgy6LF&export=download [following]
--2024-10-21 15:48:46--  https://drive.usercontent.google.com/download?id=1YcNlffQl6E09Erst__EZvlC0iJRgy6LF&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 74.125.24.132, 2404:6800:4003:c05::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|74.125.24.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20809441 (20M) [application/octet-stream]
Saving to: ‘libros.zip’

libros.zip          100%[===================>]  19.84M  58.2MB/s    in 0.3s

In [17]:
path_docs = "./libros"
os.listdir(path_docs)

['Harry Potter - Book 3 - The Prisoner of Azkaban.pdf',
 'Harry Potter - Book 1 - The Sorcerers Stone.pdf',
 'Harry Potter - Book 2 - The Chamber of Secrets.pdf',
 'Harry Potter - Book 6 - The Half-Blood Prince.pdf',
 'Harry Potter - Book 7 - The Deathly Hallows.pdf',
 'Harry Potter - Book 4 - The Goblet of Fire.pdf',
 'Harry Potter - Book 5 - The Order of the Phoenix.pdf']

In [18]:
list_pdf = [x for x in os.listdir(path_docs) if x.endswith(".pdf")]

In [10]:
chunk_size = 1000
chunk_overlap = 200

all_docs = []
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
    )

for name in tqdm(list_pdf):
  if name.endswith(".pdf"):
    path_tmp = os.path.join(path_docs, name)
    loader = PyPDFLoader(path_tmp)
    text = loader.load()
    chunks_of_text = text_splitter.split_documents(text)
    all_docs.extend(chunks_of_text)

  0%|          | 0/7 [00:00<?, ?it/s]

In [42]:
chunk = all_docs[100]
print_lines(chunk.page_content)

only ones the Trace can’t detect, because we don’t 
need to cast spells to use them: brooms, thestrals, 
and Hagrid’s motorbike.” Harry could see ﬂaws in this 
plan; however, he held his tongue to give Mad-Eye the 
chance to address them. “Now, your mother’s charm will 
only break under two condi- tions: when you come of 
age, or”—Moody gestured around the pristine kitchen—“you 
no longer call this place home. You and your aunt and 
uncle are going your separate ways tonight, in the 
full understanding that you’re never going to live 
together again, correct?” Harry nodded. “So this time, 
when you leave, there’ll be no going back, and the 
charm will break the moment you get outside its range. 
We’ve choosing to break it early, because the alternative 
is waiting for 47 


# 2. Embeddings


In [74]:
client = OpenAI(
    # This is the default and can be omitted
    api_key=api_key,
)

from openai import OpenAI
client = OpenAI()

def get_embedding(text, model="text-embedding-3-small"):
   return client.embeddings.create(input = [text], model=model).data[0].embedding

phrase = "i love soup"
embeddings = get_embedding(phrase)
print(embeddings)

[0.0003943320771213621, -0.05429684743285179, -0.050523508340120316, -0.03307795152068138, 0.022113224491477013, -0.036630768328905106, 0.0007442539208568633, 0.007179140578955412, -0.02859405241906643, -0.03523414582014084, 0.015828412026166916, -0.06037338823080063, -0.031950850039720535, 0.04491250962018967, 0.006297062151134014, 0.030186694115400314, 0.010891222395002842, 0.006682971492409706, 0.02188045345246792, 0.004888186231255531, 0.0056967586278915405, 0.0037182068917900324, 0.004272568970918655, 0.0038897220510989428, 0.01352520752698183, 0.017702829092741013, 0.006548209581524134, 0.052973728626966476, 0.020618589594960213, -0.038394927978515625, -0.016967764124274254, -0.027736475691199303, -0.040551118552684784, -0.018597159534692764, 0.019552744925022125, -0.033592499792575836, -0.0042388783767819405, 0.024318421259522438, -0.028692061081528664, -0.022529760375618935, -0.025996821001172066, -0.006376693956553936, 0.06012836471199989, -0.02005504071712494, -0.008306241594

In [75]:
embeddings = get_embedding(chunk.page_content)
print(embeddings)

[0.041026387363672256, 0.08796846121549606, -0.011694437824189663, 0.01444687508046627, -0.012194258160889149, -0.015473904088139534, -0.009393892250955105, 0.002410094952210784, -0.009537676349282265, 0.019417695701122284, 0.016692645847797394, -0.0026480231899768114, -0.01828111708164215, 0.036836110055446625, -0.01147533766925335, -0.016939133405685425, -0.035685837268829346, 0.037192147225141525, -0.005073523614555597, -0.0066380309872329235, 0.0061792912892997265, 0.03642529621720314, 0.03702782094478607, -0.02158130332827568, -0.010270290076732635, 0.009120018221437931, 0.024922572076320648, 0.011858762241899967, 0.041820622980594635, 0.03735646978020668, 0.0046147839166224, -0.01502201147377491, 0.06134786829352379, -0.02971537411212921, 0.011694437824189663, -0.0009713983163237572, -0.006908481940627098, -0.03639790788292885, 0.01331714354455471, -0.012132636271417141, 0.0010141911916434765, -0.02652473747730255, -0.013775883242487907, -0.032782766968011856, 0.02566203288733959

In [78]:
embeddings = OpenAIEmbeddings(openai_api_key=api_key,
                              model="text-embedding-3-small")


In [79]:
query_result = embeddings.embed_query(chunk.page_content)
print(query_result)

[0.041026387363672256, 0.08796846121549606, -0.011694437824189663, 0.01444687508046627, -0.012194258160889149, -0.015473904088139534, -0.009393892250955105, 0.002410094952210784, -0.009537676349282265, 0.019417695701122284, 0.016692645847797394, -0.0026480231899768114, -0.01828111708164215, 0.036836110055446625, -0.01147533766925335, -0.016939133405685425, -0.035685837268829346, 0.037192147225141525, -0.005073523614555597, -0.0066380309872329235, 0.0061792912892997265, 0.03642529621720314, 0.03702782094478607, -0.02158130332827568, -0.010270290076732635, 0.009120018221437931, 0.024922572076320648, 0.011858762241899967, 0.041820622980594635, 0.03735646978020668, 0.0046147839166224, -0.01502201147377491, 0.06134786829352379, -0.02971537411212921, 0.011694437824189663, -0.0009713983163237572, -0.006908481940627098, -0.03639790788292885, 0.01331714354455471, -0.012132636271417141, 0.0010141911916434765, -0.02652473747730255, -0.013775883242487907, -0.032782766968011856, 0.02566203288733959

# 3. Vector Store

In [80]:
print(len(all_docs))

8927


In [81]:
db = FAISS.from_documents(all_docs, embeddings)

In [82]:
question = "Who is Mr. Dursley?"
searchDocs = db.similarity_search(question)
print_lines(searchDocs[0].page_content)

As he had expected, Mrs. Dursley looked shocked and 
angry. After all, they normally pretended she didn’t 
have a sister. “No,” she said sharply. “Why?” “Funny 
stuff on the news,” Mr. Dursley mumbled. “Owls…shooting 
stars…and there were a lot of funny-looking people 
in town today.…” “So?” snapped Mrs. Dursley. “Well, 
I just thought…maybe…it was something to do with…you 
know…her crowd.” Mrs. Dursley sipped her tea through 
pursed lips. Mr. Dursley wondered whether he dared 
tell her he’d heard the name “Potter.” He decided he 
didn’t dare. Instead he said, as casually as he could, 
“Their son — he’d be about Dudley’s age now, wouldn’t 
he?” “I suppose so,” said Mrs. Dursley stiffly. “What’s 
his name again? Howard, isn’t it?” “Harry. Nasty, common 
name, if you ask me.” “Oh, yes,” said Mr. Dursley, 
his heart sinking horribly. “Yes, I quite 


## 5. RAG

In [93]:
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [94]:
llm = ChatOpenAI(model="gpt-4o-mini")
retriever = db.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [95]:
rag_chain.invoke("Who is Mr. Dursley?")

"Mr. Dursley, also known as Vernon Dursley, is Harry Potter's uncle and a Muggle. He is married to Petunia Dursley and is characterized by his disdain for anything magical. The Dursleys raised Harry after the death of his parents, hoping to suppress his magical abilities."